In [1]:
from all_func import *
from langchain_ollama import OllamaLLM, ChatOllama

In [165]:
llm = ChatOllama(model="llama3.1", temperature=0.5)

In [166]:
from langchain_core.tools import tool
from datetime import datetime

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

@tool
def get_time() -> str:
    """
    Returns the current time as a formatted string. When called, this tool will return the current time in the format "YYYY-MM-DD HH:MM:SS"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

tools = [add, multiply, get_time]

llm_with_tools = llm.bind_tools(tools)

In [119]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '03fe9be3-d1b7-432c-becc-10bf63b6ab34', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '1b0c39d3-2321-4f70-9a02-88b022f359ba', 'type': 'tool_call'}]


[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-11-03T11:14:26.967793Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply', 'arguments': {'a': 3, 'b': 12}}}, {'function': {'name': 'add', 'arguments': {'a': 11, 'b': 49}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1080986625, 'load_duration': 32799292, 'prompt_eval_count': 233, 'prompt_eval_duration': 268790000, 'eval_count': 43, 'eval_duration': 778571000}, id='run-795bf5f6-28c8-4697-a2b4-2ae5a67e8edd-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '03fe9be3-d1b7-432c-becc-10bf63b6ab34', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '1b0c39d3-2321-4f70-9a02-88b022f359ba', 'type': 'tool_call'}], usage_metadata={'input_tokens': 233, 'output_tokens': 43, 'total_tokens'

In [158]:
messages = [HumanMessage('what is 2*5')]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

llm_with_tools.invoke(messages)

[{'name': 'multiply', 'args': {'a': 2, 'b': 5}, 'id': '70247de0-f4a7-4dec-aad3-4b169e95d1f2', 'type': 'tool_call'}]


AIMessage(content='The result of the calculation 2 * 5 is 10.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-11-03T11:32:31.1234Z', 'message': {'role': 'assistant', 'content': 'The result of the calculation 2 * 5 is 10.'}, 'done_reason': 'stop', 'done': True, 'total_duration': 442258375, 'load_duration': 11963708, 'prompt_eval_count': 93, 'prompt_eval_duration': 139818000, 'eval_count': 15, 'eval_duration': 287850000}, id='run-a575bb70-2f40-491e-8cce-faf9cccba8af-0', usage_metadata={'input_tokens': 93, 'output_tokens': 15, 'total_tokens': 108})

In [167]:
import gradio as gr
from langchain.schema import AIMessage, HumanMessage

def predict(message, history):
    messages = [HumanMessage(message)]

    ai_msg = llm_with_tools.invoke(messages)

    messages.append(ai_msg)

    for tool_call in ai_msg.tool_calls:
        selected_tool = {"add": add, "multiply": multiply, "get_time": get_time}[tool_call["name"].lower()]
        tool_msg = selected_tool.invoke(tool_call)
        messages.append(tool_msg)
    
    return llm_with_tools.invoke(messages).content

gr.ChatInterface(predict, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7892

To create a public link, set `share=True` in `launch()`.


In [84]:
from langchain_core.messages import ToolMessage
def predict(message, history):
    history_langchain_format = []
    messages = [HumanMessage(content=message)]
    for msg in history:
        if msg.type == "human":
            history_langchain_format.append(HumanMessage(content=msg.content))
        elif msg.type == "ai":
            history_langchain_format.append(AIMessage(content=msg.content))
        elif msg.type == "tool":
            history_langchain_format.append(ToolMessage(content=msg.content))
    
    ai_msg = llm_with_tools.invoke(message)
    messages.append(ai_msg)

    for tool_call in ai_msg.tool_calls:
        selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
        tool_msg = selected_tool.invoke(tool_call)
        messages.append(tool_msg)

    history_langchain_format += messages
    return history_langchain_format

In [85]:
test = predict("What is 3 * 12? Also, what is 11 + 49?", [])
test.append(llm_with_tools.invoke(test))


In [87]:
predict("What is 3 * 12? Also, what is 11 + 49?", test)

KeyError: 'tool_call_id'

In [130]:
get_current_time.invoke({})

'2024-11-03 03:17:05'